Copyright (c) 2024-2025 Ken Barker

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Calculate Geodesic Inverse Initial Azimuth

Calculate the intial azimuth of a geodesic between two positions and show the number of iterations required.

This notebook compares the original version of `find_azimuth_and_aux_length`, which uses the great circle azimuth
on the auxiliary sphere with the new version which estimates the geodesic azimuth from the longitude difference
between the auxiliary sphere and the ellipsoid.

Performs calculations and compares the results with data from Charles Karney's [Test data for geodesics](https://geographiclib.sourceforge.io/C++/doc/geodesic.html#testgeod).

The contents of the data file are as follows:

- 0-100000 entries randomly distributed
- 100000-150000 entries which are nearly antipodal
- 150000-200000 entries with short distances
- 200000-250000 entries with one end near a pole
- 250000-300000 entries with both ends near opposite poles
- 300000-350000 entries which are nearly meridional
- 350000-400000 entries which are nearly equatorial
- 400000-450000 entries running between vertices (α1 = α2 = 90°)
- 450000-500000 entries ending close to vertices

In [ ]:
%matplotlib inline
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
from scipy import stats
from os import environ, path
from enum import Enum

from via_angle import Angle, Degrees, Radians
from via_sphere import LatLong, MIN_VALUE
from via_units import Metres
from via_ellipsoid import Ellipsoid, aux_sphere_azimuth_length

## Read the GeodTest.dat file into a polars LazyFrame

In [ ]:
# The columns of the GeodTest.dat file
class Column(Enum):
    latitude_1   = 0
    longitude_1  = 1
    azimuth_1    = 2
    latitude_2   = 3
    longitude_2  = 4
    azimuth_2    = 5
    distance_m   = 6
    distance_deg = 7
    m12          = 8
    area         = 9

# Read the geodesic test data file into a polars LazyFrame: lf
TEST_DIRECTORY_ENV = 'GEODTEST_DIR'
TEST_FILENAME = 'GeodTest.dat'
test_directory = environ.get(TEST_DIRECTORY_ENV)
filename = path.join(test_directory, TEST_FILENAME)
# Select all the test data entries
start = 0
size = 500000
lf = pl.scan_csv(filename, separator=' ', has_header=False).select(
    ['column_1', 'column_2', 'column_3', 'column_4', 'column_5']
).slice(start, size).collect()
lf.schema

### Calculate geodesic azimuths and iterations

In [ ]:
%%time
delta_azimuth_org = np.empty(size)
delta_azimuth_new = np.empty(size)
iterations_org = np.empty(size)
iterations_new = np.empty(size)

i = 0
for row in lf.rows():
    # Get departure and arrival positions from lf
    lat_1 = Angle(Degrees(row[Column.latitude_1.value]))
    lat_2 = Angle(Degrees(row[Column.latitude_2.value]))
    delta_long = Angle(Degrees(row[Column.longitude_2.value]))

    # calculate parametric latitudes from geodetic latitudes
    beta1 = Ellipsoid.wgs84().calculate_parametric_latitude(lat_1)
    beta2 = Ellipsoid.wgs84().calculate_parametric_latitude(lat_2)

    # solve the inverse geodesic problem on the auxiliary sphere
    azim_aux, length_aux, iters_aux = aux_sphere_azimuth_length(beta1, beta2, delta_long, Ellipsoid.wgs84(), Radians(MIN_VALUE), False)
    delta_azimuth_org[i] = np.abs(azim_aux.to_degrees().v() - row[Column.azimuth_1.value])
    iterations_org[i] = iters_aux

    # solve the inverse geodesic problem on the auxiliary sphere, estimating the initial azimuth
    azim_aux, length_aux, iters_aux = aux_sphere_azimuth_length(beta1, beta2, delta_long, Ellipsoid.wgs84(), Radians(MIN_VALUE), True)
    delta_azimuth_new[i] = np.abs(azim_aux.to_degrees().v() - row[Column.azimuth_1.value])
    iterations_new[i] = iters_aux
    
    i += 1

In [ ]:
stats.describe(delta_azimuth_org)

In [ ]:
stats.describe(delta_azimuth_new)

In [ ]:
stats.describe(iterations_org)

In [ ]:
stats.describe(iterations_new)

In [ ]:
fig, axes = plt.subplots(2, 1, sharey=True)

axes[0].hist(iterations_org, bins=np.arange(8), align='left', color='#808080')
axes[0].set_ylabel('Original Samples')

axes[1].hist(iterations_new, bins=np.arange(8), align='left', color='#000080')
axes[1].set_xlabel('Iterations')
axes[1].set_ylabel('New Samples')
# plt.savefig('initial_azimuth_iterations.svg')

The graphs above show that the new algorithm, estimating the geodesic azimuth, converges quicker
(i.e. in less iterations) than the original algorithm.